In [1]:
import pandas as pd

data = {
    'OrderID': [101, 102, 103, 104, 105, 106],
    'Customer': ['A', 'B', 'A', 'C', 'B', 'A'],
    'Category': ['Electronics', 'Clothing', 'Electronics', 'Grocery', 'Clothing', 'Grocery'],
    'Amount': [2500, 800, 3000, 150, 900, 200],
    'OrderDate': ['2024-01-10', '2024-01-11', '2024-01-10', '2024-01-12', '2024-01-11', '2024-01-13']
}

df = pd.DataFrame(data)
df['OrderDate'] = pd.to_datetime(df['OrderDate'])


In [2]:
df

,OrderID,Customer,Category,Amount,OrderDate
0,101,A,Electronics,2500,2024-01-10
1,102,B,Clothing,800,2024-01-11
2,103,A,Electronics,3000,2024-01-10
3,104,C,Grocery,150,2024-01-12
4,105,B,Clothing,900,2024-01-11
5,106,A,Grocery,200,2024-01-13


--------
### **1. How do you find the total amount spent by each customer?**

In [4]:
df.groupby("Customer")["Amount"].sum().to_frame()

,Amount
Customer,
A,5700
B,1700
C,150


-------

### **2. How do you get the number of orders for each category?**

In [6]:
df.groupby("Category")["OrderID"].count().to_frame()

,OrderID
Category,
Clothing,2
Electronics,2
Grocery,2


--------

### **3. How do you filter all orders where the amount is above the average order amount?**

In [11]:
df[df["Amount"] > df["Amount"].mean()]

,OrderID,Customer,Category,Amount,OrderDate
0,101,A,Electronics,2500,2024-01-10
2,103,A,Electronics,3000,2024-01-10


-------

### **4. How do you find the order with the maximum amount for each customer?**

In [15]:
df.loc[df.groupby('Customer')['Amount'].idxmax()]

,OrderID,Customer,Category,Amount,OrderDate
2,103,A,Electronics,3000,2024-01-10
4,105,B,Clothing,900,2024-01-11
3,104,C,Grocery,150,2024-01-12


-------

### **5. How do you add a new column indicating whether the order amount is “High” (>1000) or “Low”?**

In [17]:
import numpy as np
df["New_Column"] = np.where(df["Amount"] > 1000, "High", "Low")

In [18]:
df

,OrderID,Customer,Category,Amount,OrderDate,New_Column
0,101,A,Electronics,2500,2024-01-10,High
1,102,B,Clothing,800,2024-01-11,Low
2,103,A,Electronics,3000,2024-01-10,High
3,104,C,Grocery,150,2024-01-12,Low
4,105,B,Clothing,900,2024-01-11,Low
5,106,A,Grocery,200,2024-01-13,Low


-------
### **6. How do you extract only those rows where the order date is a weekend?**

In [22]:
df[(df["OrderDate"].dt.day_name() == "Saturday") | (df["OrderDate"].dt.day_name() == "Sunday")]

,OrderID,Customer,Category,Amount,OrderDate,New_Column
5,106,A,Grocery,200,2024-01-13,Low


In [23]:
df[df["OrderDate"].dt.dayofweek >= 5]

,OrderID,Customer,Category,Amount,OrderDate,New_Column
5,106,A,Grocery,200,2024-01-13,Low


-------
### **7. How do you find the cumulative amount spent by each customer based on order date?**

In [26]:
df.sort_values(["Customer", "OrderDate"], inplace=True)

df["Cumulative_Amount"] = df.groupby("Customer")["Amount"].cumsum()
df

,OrderID,Customer,Category,Amount,OrderDate,New_Column,Cumulative_Amount
0,101,A,Electronics,2500,2024-01-10,High,2500
2,103,A,Electronics,3000,2024-01-10,High,5500
5,106,A,Grocery,200,2024-01-13,Low,5700
1,102,B,Clothing,800,2024-01-11,Low,800
4,105,B,Clothing,900,2024-01-11,Low,1700
3,104,C,Grocery,150,2024-01-12,Low,150


------

### **8. How do you pivot the table so that categories become columns and values show total amount?**

In [32]:
df.pivot_table(
    index="Customer",
    columns="Category",
    values="Amount",
    aggfunc="sum"
)


Category,Clothing,Electronics,Grocery
Customer,,,
A,NaN,5500.0,200.0
B,1700.0,NaN,NaN
C,NaN,NaN,150.0


-------
### **9. How do you rank each customer’s orders based on amount (highest = rank 1)?**

In [31]:
df.sort_values(by = ["Customer", "Amount"], ascending = [True, False])

df['rank_column'] = df.groupby("Customer")['Amount'].rank(method='dense', ascending=False)
df

,OrderID,Customer,Category,Amount,OrderDate,New_Column,Cumulative_Amount,rank_column
0,101,A,Electronics,2500,2024-01-10,High,2500,2.0
2,103,A,Electronics,3000,2024-01-10,High,5500,1.0
5,106,A,Grocery,200,2024-01-13,Low,5700,3.0
1,102,B,Clothing,800,2024-01-11,Low,800,2.0
4,105,B,Clothing,900,2024-01-11,Low,1700,1.0
3,104,C,Grocery,150,2024-01-12,Low,150,1.0


-------
### **10. How do you merge this df with another df (say customer details) and keep only matching rows?**

In [33]:
df1 = pd.DataFrame({
    "Customer": ["A", "B", "C"],
    "City": ["Bengaluru", "Mumbai", "Chennai"],
    "Membership": ["Gold", "Silver", "Bronze"]
})


In [34]:
df

,OrderID,Customer,Category,Amount,OrderDate,New_Column,Cumulative_Amount,rank_column
0,101,A,Electronics,2500,2024-01-10,High,2500,2.0
2,103,A,Electronics,3000,2024-01-10,High,5500,1.0
5,106,A,Grocery,200,2024-01-13,Low,5700,3.0
1,102,B,Clothing,800,2024-01-11,Low,800,2.0
4,105,B,Clothing,900,2024-01-11,Low,1700,1.0
3,104,C,Grocery,150,2024-01-12,Low,150,1.0


In [35]:
df1

,Customer,City,Membership
0,A,Bengaluru,Gold
1,B,Mumbai,Silver
2,C,Chennai,Bronze


In [36]:
df.merge(df1, on = "Customer", how = "inner")

,OrderID,Customer,Category,Amount,OrderDate,New_Column,Cumulative_Amount,rank_column,City,Membership
0,101,A,Electronics,2500,2024-01-10,High,2500,2.0,Bengaluru,Gold
1,103,A,Electronics,3000,2024-01-10,High,5500,1.0,Bengaluru,Gold
2,106,A,Grocery,200,2024-01-13,Low,5700,3.0,Bengaluru,Gold
3,102,B,Clothing,800,2024-01-11,Low,800,2.0,Mumbai,Silver
4,105,B,Clothing,900,2024-01-11,Low,1700,1.0,Mumbai,Silver
5,104,C,Grocery,150,2024-01-12,Low,150,1.0,Chennai,Bronze


----------------------------------